<a href="https://colab.research.google.com/github/em-bst/enhancing-large-language-models-with-human-feedback/blob/main/RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install trl
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### Reward model training configuration

In [ ]:
""" Full training:
    --model_name_or_path gpt2 \
    --dataset_name trl-lib/ultrafeedback_binarized \
    --output_dir ./drive/reward_output \
    --per_device_train_batch_size 4 \
    --num_train_epochs 1 \
    --learning_rate 1.0e-5 \
    --logging_steps 25 \
    --eval_strategy steps \
    --eval_steps 50 \
    --max_length 1024 """

' Full training:\npython examples/scripts/reward_modeling.py     --model_name_or_path gpt2     --dataset_name trl-lib/ultrafeedback_binarized     --output_dir ./drive/reward_output     --per_device_train_batch_size 4     --num_train_epochs 1     --gradient_checkpointing True     --learning_rate 1.0e-5     --logging_steps 25     --eval_strategy steps     --eval_steps 50     --max_length 2048 '

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Loading of the libraries:

In [ ]:
import warnings
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, HfArgumentParser
from datasets import load_dataset, DatasetDict, load_from_disk

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    setup_chat_format,
)

#### GPUs' Configuration

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#### Configuration of the reward model

In [ ]:
model_name_ = "gpt2"
output_dir = "./drive/reward_output"
dataset_name = "trl-lib/ultrafeedback_binarized"


In [ ]:
model_args = ModelConfig(model_name_or_path=model_name_)


In [ ]:
model_args

ModelConfig(model_name_or_path='gpt2', model_revision='main', torch_dtype=None, trust_remote_code=False, attn_implementation=None, use_peft=False, lora_r=16, lora_alpha=32, lora_dropout=0.05, lora_target_modules=None, lora_modules_to_save=None, lora_task_type='CAUSAL_LM', use_rslora=False, load_in_8bit=False, load_in_4bit=False, bnb_4bit_quant_type='nf4', use_bnb_nested_quant=False)

In [ ]:
# Definition of the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, trust_remote_code=model_args.trust_remote_code, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Definition of the Model
model = AutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path, num_labels=1, trust_remote_code=model_args.trust_remote_code)
model.config.pad_token_id = tokenizer.pad_token_id
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=1, bias=False)
)

#### Loading of the dataset : ultrafeedback_binarized from the trl library

In [ ]:
dataset = load_dataset("trl-lib/ultrafeedback_binarized")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 62135
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 1000
    })
})


#### Data Preparation ie. tokenization of the data.
The goal is to tokenize the "chosen" and "rejected" text columns and add the resulting tokenized information as new columns (input_ids_chosen, attention_mask_chosen, input_ids_rejected, attention_mask_rejected) to the dataset. These tokenized columns are crucial for training, as they allow the model to process the text inputs efficiently in the training steps.

In [ ]:
def tokenize_function(batch):
    """
    Tokenizes and preprocesses the input examples.

    Args:
        examples: A dictionary of examples containing the "chosen" key.

    Returns:
        A dictionary containing the tokenized input IDs, attention mask, etc.
    """
    return tokenizer(
        [(str(chosen), str(rejected)) for (chosen, rejected) in zip(batch["chosen"], batch["rejected"])],
        truncation=True,  # Truncate sequences longer than max_length
        padding="max_length",  # Pad shorter sequences to max_length
        max_length=min(tokenizer.model_max_length, 200),  # Set max sequence length (reduced for CUDA memory)
        return_tensors="pt"  # Return PyTorch tensors
    )

# Apply the tokenize_function to the dataset using the `map` function
tokenized_dataset = dataset.map(
    tokenize_function,  # The function to apply
    batched=True,  # Process examples in batches for efficiency
  # Use 4 processes for parallel processing
)

Map:   0%|          | 0/62135 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.save_to_disk('/content/drive/MyDrive/tokenized_datasets')

Saving the dataset (0/5 shards):   0%|          | 0/62135 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

#### Taking a subset of the training data for computation and memory purposes

In [ ]:
train_subset_indices = torch.randint(0, len(tokenized_dataset["train"]), size=(int(0.02 * len(tokenized_dataset["train"])),))
train_subset = tokenized_dataset["train"].select(train_subset_indices.tolist())

# Update the dataset with the new subset
tokenized_dataset['train'] = train_subset

The dataset is now made of 1,242 training points and 1,000 testing ones.

Training Parameters 🚀

In [ ]:
if tokenizer.chat_template is None:
        model, tokenizer = setup_chat_format(model, tokenizer)

script_args = ScriptArguments(
    dataset_name=dataset_name,
    dataset_train_split="train",
    dataset_test_split="test")

training_args = RewardConfig(
    output_dir="/content/drive/MyDrive/reward_output",           # Répertoire de sortie
    per_device_train_batch_size=8,               # Taille de batch par périphérique
    num_train_epochs=1,                          # Nombre d'époques,                 # Activer Gradient Checkpointing
    learning_rate=1.0e-5,                        # Taux d'apprentissage
    logging_steps=25,                            # Intervalle pour afficher les logs
    evaluation_strategy="steps",                 # Évaluer à chaque `eval_steps`
    eval_steps=50,# Intervalle pour évaluation                             # Pas de limite explicite sur les étapes
    report_to = "none",
    dataset_num_proc=4
)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


#### Reward Model Training⚡

In [ ]:
##########
# Training
##########
trainer = RewardTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset[script_args.dataset_train_split],
    eval_dataset=tokenized_dataset[script_args.dataset_test_split],
    )
trainer.train()

Map:   0%|          | 0/1242 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1242 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1210 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1074 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1054 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1025 > 1024). Running this sequence through the model will result in indexing errors


Filter (num_proc=4):   0%|          | 0/1242 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
50,0.819300,0.693147,0.508824


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|im_start|>user                              │ <|im_start|>user                             │ [0.4218, 0.5782] │
│ Definition: In this task, you are given a     │ Definition: In this task, you are given a    │                  │
│ summary for US Congressional and California   │ summary for US Congressional and California  │                  │
│ state bill, your task is to generate a Title  │ state bill, your task is to generate a Title │                  │
│ for this bill. The preferred titles are under │ for this bill. The preferred titles are      │                  │
│ forty words and mention the purpose of the    │ under forty words and mention the purpose of │                  │
│ bill.                                         │ the bill.                                    │                  │
│ Input: Intelligent Technologies Initiative    │ Input: Intelligent Technologies Initiative   │                  │
│ Act of 2015  This bill directs the Department │ Act of 2015  This bill directs the           │                  │
│ of Transportation (DOT) to establish an       │ Department of Transportation (DOT) to        │                  │
│ Intelligent Technology Initiative to provide  │ establish an Intelligent Technology          │                  │
│ grants to up to six state and local           │ Initiative to provide grants to up to six    │                  │
│ governments to establish deployment sites for │ state and local governments to establish     │                  │
│ large scale installation and operation of     │ deployment sites for large scale             │                  │
│ intelligent transportation systems (ITS) to   │ installation and operation of intelligent    │                  │
│ improve safety, efficiency, system            │ transportation systems (ITS) to improve      │                  │
│ performance, and return on investment. DOT    │ safety, efficiency, system performance, and  │                  │
│ shall develop criteria for selecting grant    │ return on investment. DOT shall develop      │                  │
│ recipients, including how the deployment of   │ criteria for selecting grant recipients,     │                  │
│ technology will enable the recipient to:      │ including how the deployment of technology   │                  │
│ reduce costs and improve return on            │ will enable the recipient to:   reduce costs │                  │
│ investments;   deliver environmental benefits │ and improve return on investments;   deliver │                  │
│ and reduce energy consumption by alleviating  │ environmental benefits and reduce energy     │                  │
│ congestion and streamlining traffic flow;     │ consumption by alleviating congestion and    │                  │
│ measure and improve the operational           │ streamlining traffic flow;   measure and     │                  │
│ performance of its transportation network;    │ improve the operational performance of its   │                  │
│ reduce the number and severity of traffic     │ transportation network;   reduce the number  │                  │
│ collisions and increase driver, passenger,    │ and severity of traffic collisions and       │                  │
│ and pedestrian safety;  collect, disseminate, │ increase driver, passenger, and pedestrian   │                  │
│ and utilize real-time traffic, transit,       │ safety;  collect, disseminate, and utilize   │                  │
│ parking, and other transportation-related     │ real-time traffic, transit, parking, and     │                  │
│ information to improve mobility, reduce       │ other 

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:785: UserWarning: There are 210 out of 550 instances where the predictions for both options are equal. These instances are ignored in the accuracy computation.
  warnings.warn(


TrainOutput(global_step=86, training_loss=0.8161915956541549, metrics={'train_runtime': 165.1817, 'train_samples_per_second': 4.153, 'train_steps_per_second': 0.521, 'total_flos': 0.0, 'train_loss': 0.8161915956541549, 'epoch': 1.0})

#### Saving the training parameters and the model

In [ ]:
trainer.save_model(training_args.output_dir)
model.save_pretrained("drive/MyDrive/reward_output")